In [25]:
import pandas as pd

In [55]:
fname = '../../CVGFvenn_geneID/WT_only.txt'
df = pd.read_csv(fname, sep='\t',header=None)
CVonlyGenes = df[0].tolist()

In [64]:
def ProcessMeta2dResults(group):
    WT = pd.read_csv(f"../meta2d_{group}_WT.txt",sep="\t",index_col='CycID').iloc[:,15:] #only keep the meta2d columns
    KO = pd.read_csv(f"../meta2d_{group}_KO.txt",sep="\t",index_col='CycID').iloc[:,15:] 
    WT.columns = [f"WT_{i}" for i in WT.columns]
    KO.columns = [f"KO_{i}" for i in KO.columns]
    WTKO = pd.concat([WT,KO],axis=1)

    WTKO = WTKO.loc[CVonlyGenes] #only keep the CV oscillating genes

    WTKO['rAMP_meta2d_WTdivKO'] = WTKO['WT_meta2d_rAMP']/WTKO['KO_meta2d_rAMP']
    WTKO['rAMP_meta2d_WTsubKO'] = WTKO['WT_meta2d_rAMP']-WTKO['KO_meta2d_rAMP']
    WTKO['phase_meta2d_abs_WTsubKO'] = abs(WTKO['WT_meta2d_phase']-WTKO['KO_meta2d_phase'])
    #update phase difference value to be between 0 and 12 (in 24 h day/light cycle) 
    #if x is > 12, then x = 24-x
    filt = WTKO['phase_meta2d_abs_WTsubKO']>12
    WTKO.loc[filt,'phase_meta2d_abs_WTsubKO'] = 24-WTKO.loc[filt,'phase_meta2d_abs_WTsubKO']

    WTKO.to_csv(f"meta2d_{group}.csv",sep=",")

groups = ['nr1d1','hdac3','nfil3']

for group in groups:
    ProcessMeta2dResults(group)